In [1]:
using PyCall
cma = pyimport("cma")

PyObject <module 'cma' from '/home/mkoehler/.local/lib/python3.8/site-packages/cma/__init__.py'>

In [2]:
using DrWatson
@quickactivate :Catalyst
using Distributed
N = 12
addprocs(N)

┌ Warning: /home/mkoehler/.julia/packages/Plots/cc8wh/src/Plots.jl/ is not an existing directory, Revise is not watching
└ @ Revise /home/mkoehler/.julia/packages/Revise/Pcs5V/src/Revise.jl:489


12-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13

In [3]:
@everywhere using DrWatson
@everywhere @quickactivate :Catalyst

      From worker 4:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 9:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 13:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 6:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 12:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 11:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 7:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 8:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 2:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.toml`
      From worker 5:	Activating environment at `~/Repos/advection-diffusion-catalysis/Project.tom

In [5]:
function calibrate(cma,N)
    Init = [0.013235189689493238, 1., 1., 1., 10.] #initial guess
    opts = cma.CMAOptions()
    opts["bounds"] = [[0, 0, 0, 0, 0], [1, 1, 3, 1000, 1000]] 
    opts["popsize"] = N
    weights = cma.recombination_weights.RecombinationWeights(N)
    opts["CMA_recombination_weights"] = weights
    es = cma.CMAEvolutionStrategy(Init, 0.5, opts) 
    
    input_exp = []
    output_exp = []
    for row in CSV.File(datadir("experiment/SBA15_interp_10.csv"); delim = ",")
        push!(input_exp, row.I)
        push!(output_exp, row.O)
    end
    n = 2
    nels = (5n, convert(Int,2.5n),n)
    S = Vec((0.0,0.0,0.0))
    E = Vec((1.0,0.5,0.2))
    grid = generate_grid(Tetrahedron, nels, S, E)
    faces = union(getfaceset(grid,"top"),getfaceset(grid,"bottom"),getfaceset(grid,"left"),getfaceset(grid,"right"),
                   getfaceset(grid,"front"),getfaceset(grid,"back"))
    addfaceset!(grid, "1", faces)
    
    while isempty(es.stop()) 
        solutions = es.ask() 
        fitness = pmap(function eval(input_args) 
                            try 
                               return Catalyst.solve(input_args[1], input_args[2]*1e-1, 1.0, input_args[3], 
                                          input_exp, output_exp, progress=false, 
                                          microcomp_type=:linear, 
                                          Q=input_args[4], kₙ=input_args[5], 
                                          calibration=true, micromesh=grid)#, w=2.2954e-4*(1/0.37))
                             catch err #in case we loose newton convergence in a microscopic computation 
                                return 1e10
                             end
                        end
                            ,solutions) 
        es.tell(solutions,fitness) 
        es.logger.add() 
        es.disp() 
    end
end

calibrate (generic function with 1 method)

In [ ]:
calibrate(cma, N)

(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 5 (seed=763990, Wed Sep  9 10:25:50 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     12 3.522309532184408e+02 1.0e+00 4.32e-01  4e-01  4e-01 10:35.4
    2     24 3.522508284620995e+02 1.3e+00 3.91e-01  3e-01  4e-01 19:57.4
    3     36 3.523735262432452e+02 1.5e+00 3.73e-01  2e-01  4e-01 29:17.0
    4     48 3.523198142478491e+02 1.8e+00 3.58e-01  2e-01  4e-01 38:30.1
    5     60 3.523159445805497e+02 2.0e+00 3.21e-01  2e-01  3e-01 47:48.7
    6     72 3.523190344761943e+02 2.2e+00 3.11e-01  1e-01  3e-01 57:03.9
    7     84 3.521551967531026e+02 2.7e+00 3.96e-01  2e-01  5e-01 66:20.7
    8     96 3.523192522284052e+02 3.3e+00 3.54e-01  1e-01  4e-01 75:31.9
    9    108 3.523185935169642e+02 3.8e+00 3.03e-01  1e-01  4e-01 84:45.3
